In [1]:
import yfinance as yf
import numpy as np
import pandas as pd
from statsmodels.tsa.stattools import adfuller
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from datetime import datetime, timedelta
from statsmodels.tsa.arima.model import ARIMA

In [2]:
def get_data(ticker):
    stock = yf.Ticker(ticker)
    stock_data = stock.history(start="2024-01-01")
    return stock_data

In [3]:
def stationary_check(close_price):
    # Ensure 1D input
    if isinstance(close_price, pd.DataFrame):
        close_price = close_price.iloc[:, 0]
    elif isinstance(close_price, np.ndarray):
        close_price = close_price.flatten()
    
    adf_test = adfuller(close_price)
    p_value = adf_test[1]
    if p_value < 0.05:
        conclusion = 'Given data is Stationary'
    else:
        conclusion = 'Given data is not Stationary'
    return p_value, conclusion

In [4]:
def get_rolling_mean(close_price):
    if isinstance(close_price, pd.DataFrame):
        close_price = close_price["Close"]
    rolling_price = close_price.rolling(window=7).mean().dropna()
    return rolling_price


In [5]:
def get_differencing_order(close_price):
    p_value = stationary_check(close_price)[0]
    d = 0
    while True:
        if p_value > 0.05:
            d = d + 1
            close_price = close_price.diff().dropna()
            p_value = stationary_check(close_price)[0]
        else:
            break
    
    return d

In [6]:
def fit_model(data, differencing_order):
    model = ARIMA(data, order = (30,differencing_order, 30))
    model_fit = model.fit()

    forecast_steps = 30
    forecast = model_fit.get_forecast(steps=forecast_steps)
    predictions = forecast.predicted_mean
    return predictions

In [7]:
def evaluate_model(original_price, differencing_order):
    train_data, test_data = original_price[:-30], original_price[-30:]
    predictions = fit_model(train_data, differencing_order)
    rmse = np.sqrt(mean_squared_error(test_data, predictions))
    return round(rmse, 2)

In [8]:
def scaling(close_price):
    scale = StandardScaler()
    scaled_data = scale.fit_transform(np.array(close_price).reshape(-1,1))
    return scaled_data, scale

In [9]:
def get_forecast(original_price, differencing_order):
    predictions = fit_model(original_price, differencing_order)
    start_date = datetime.now().strftime("%Y-%m-%d")
    end_date = (datetime.now() + timedelta(days=29)).strftime("%Y-%m-%d")
    forecast_index = pd.date_range(start=start_date, end=end_date)
    forecast_df = pd.DataFrame(predictions, index=forecast_index, columns=["Forecast"])
    return forecast_df

In [10]:
def inverse_scaling(scale, scaled_data):
    close_price = scale.inverse_transform(np.array(scaled_data).reshape(-1, 1))
    return close_price

In [11]:
ticker = "TSLA"
data = get_data(ticker)
close_price = data["Close"]

In [12]:
rolling_mean = get_rolling_mean(close_price)
differencing_order = get_differencing_order(close_price)
scaled_data, scaler = scaling(rolling_mean)

In [ ]:
rmse = evaluate_model(scaled_data, differencing_order)
rmse

/opt/anaconda3/envs/ts/lib/python3.10/site-packages/statsmodels/tsa/statespace/sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
/opt/anaconda3/envs/ts/lib/python3.10/site-packages/statsmodels/tsa/statespace/sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
/opt/anaconda3/envs/ts/lib/python3.10/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


In [ ]:
forecast = get_forecast(scaled_data, differencing_order)
print(forecast)

/opt/anaconda3/envs/ts/lib/python3.10/site-packages/statsmodels/tsa/statespace/sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
/opt/anaconda3/envs/ts/lib/python3.10/site-packages/statsmodels/tsa/statespace/sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
/opt/anaconda3/envs/ts/lib/python3.10/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


            Forecast
2025-05-15  0.811841
2025-05-16  0.915624
2025-05-17  1.005096
2025-05-18  1.073304
2025-05-19  1.108384
2025-05-20  1.108695
2025-05-21  1.092607
2025-05-22  1.082297
2025-05-23  1.098509
2025-05-24  1.106610
2025-05-25  1.121648
2025-05-26  1.146252
2025-05-27  1.178272
2025-05-28  1.190971
2025-05-29  1.206090
2025-05-30  1.199172
2025-05-31  1.200692
2025-06-01  1.195217
2025-06-02  1.181319
2025-06-03  1.175255
2025-06-04  1.187209
2025-06-05  1.204638
2025-06-06  1.231604
2025-06-07  1.262164
2025-06-08  1.290917
2025-06-09  1.312099
2025-06-10  1.320203
2025-06-11  1.319155
2025-06-12  1.306044
2025-06-13  1.285153
